#**Машинное обучение ИБ-2024**

#**Домашнее задание 1.**
#Регрессия, KNN, LinearRegression.

В данной домашней работе мы будем строить модели для предсказания цены квартиры в России. Ниже приведено описание некоторых колонок набора данных.

date - дата публикации объявления

price - цена в рублях

level- этаж, на котором находится квартира

levels - количество этажей в квартире

rooms - количество комнат в квартире. Если значение -1, то квартира считается апартаментами.

area - площадь квартиры.

kitchen_area - площадь кухни.

geo_lat - Latitude

geo_lon - Longitude

building_type - материал застройки. 0 - Don't know. 1 - Other. 2 - Panel. 3 - Monolithic. 4 - Brick. 5 - Blocky. 6 - Wooden

#Часть 0. Начало работы

Для начала работы с данными импортируем библиотеки, которые понадобятся в данном задании.

In [ ]:
import math
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

Загрузим библиотеку folium для отображения данных на карте по координатам.

In [ ]:
!pip install folium

Распакуем наши данные из архива.

In [ ]:
!unzip ...

Загрузим данные из csv файла в датафрейм.

In [ ]:
df = pd.read_csv("input_data_first1000.csv",delimiter=";")

Отобразим на карте координаты наших построек.

In [ ]:
import folium
from IPython.display import display

map_df = df.loc[:1000]

m = folium.Map(location=[55.751244, 37.618423], zoom_start=10)

# Список точек с широтой и долготой
lats = map_df['geo_lat'].loc[:1000]
longs = map_df['geo_lon'].loc[:1000]
# Добавляем точки на карту
for point in zip(lats, longs):
    folium.Marker(
        location=[point[0], point[1]]
    ).add_to(m)

display(m)

# Часть 1. Подготовим данные для обработки моделями машинного обучения.

**0.5 Балл**. География наших наблюдений в наборе данных крайне большая. Однако мы знаем, что стоимость квартир в Москве и Санкт-Петербурге намного выше, чем в среднем по России. Давайте сделаем признаки, который показывают, находится ли квартира в 20 килиметрах от центра Москвы или находится ли квартира в 20 килиметрах от центра Санкт-Петербурга.

Создайте два признака is_Moscow и is_Saint_Peterburg. Для нахождения расстояния по координатам используйте функцию haversine_distance.

In [ ]:
# Функция для расчета дистанции по формуле Хаверсина
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Радиус Земли в километрах
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)

    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(delta_lambda / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

# Центры Москвы и Санкт-Петербурга
moscow_coords = (55.7558, 37.6173)  # latitude, longitude
spb_coords = (59.9343, 30.3351)      # latitude, longitude

# Создание признаков is_Moscow и is_Saint_Peterburg
df['is_Moscow'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], *moscow_coords) <= 20, axis=1)
df['is_Saint_Peterburg'] = df.apply(lambda row: haversine_distance(row['geo_lat'], row['geo_lon'], *spb_coords) <= 20, axis=1)


In [ ]:

backup1 = pd.HDFStore('backup1.h5')
backup1['df'] = df
backup1.close()

**0.5 Балла**. В нашем наборе данных есть признаки, которые мы теоретически можем использовать, например postal_code, но мы это будем делать в рамках домашней работы очень-очень долго. Поэтому предлагается удалить ненужные признаки из датафрейма.

Удалим geo_lat,	geo_lon,	object_type,	postal_code,	street_id,	id_region,	house_id.

In [ ]:
backup1 = pd.HDFStore('backup1.h5')
df = backup1['df']
backup1.close()

In [ ]:
df = df.drop(columns=['geo_lat', 'geo_lon', 'object_type', 'postal_code', 'street_id', 'id_region', 'house_id'])

**0.5 Балл**. Для начала Вам предлагается проанализировать Ваши оставшиеся признаки (колонки) в наборе данных. Какие колонки категориальные? Какие числовые?

Категориальные: building_type

Числовые: date, price, level, levels, rooms, area, kitchen_area

Давайте закодируем категориальные признаки с помощью OneHot-Encoding. https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Инициализация OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

# Применение OneHotEncoder к колонке building_type
encoded_df = pd.DataFrame(encoder.fit_transform(df[['building_type']]))


In [ ]:

# Создание датафрейма с кодированными значениями
encoded_df.columns = encoder.get_feature_names_out()
#encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['building_type']))

# Объединение с оригинальным датафреймом
df = df.join(encoded_df)
df.drop('building_type', axis = 1, inplace = True)
#df_combined = pd.concat([df, encoded_df], axis=1).drop(columns='building_type')


In [ ]:
# Вывод результирующего датафрейма
print(df[:15])

          date     price  level  levels  rooms    area  kitchen_area  \
0   2021-01-01   2451300     15      31      1   30.30           0.0   
1   2021-01-01   1450000      5       5      1   33.00           6.0   
2   2021-01-01  10700000      4      13      3   85.00          12.0   
3   2021-01-01   3100000      3       5      3   82.00           9.0   
4   2021-01-01   2500000      2       3      1   30.00           9.0   
5   2021-01-01   1450000      5       5      2   47.00           6.0   
6   2021-01-01   9000000      2       4      3  107.40          21.3   
7   2021-01-01   2990000      1       2      3   54.00           7.0   
8   2021-01-01   2300000     16      18      1   39.70          11.5   
9   2021-01-01   2290000      2       2      2   53.20          16.0   
10  2021-01-01   3350000      1      19     -1   10.10           2.0   
11  2021-01-01   5000000      8       9      3  100.00           0.0   
12  2021-01-01   3214804      5      17     -1   32.44          

**0.5 Балл**. Поработаем с числовыми признаками:


1.   Добавьте в ваш датасет два признака: количество дней со дня первого наблюдения (разница между датами объявлений). Возможно, для предсказания цены не так важен этаж, как важно отношение этажа квартиры на количество этажей в доме, добавьте этот признак. После добавления нового признака колонку date можно удалить.
2.   Числовые признаки могут иметь разные порядки. Давайте отнормируем числовые признаки с помощью StandartScaller https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html.



In [ ]:
## Добавляем первый признак

# Преобразование поля date в формат даты
df['date'] = pd.to_datetime(df['date'])

# Находим первую дату
first_date = df['date'].min()

# Вычисляем количество дней со дня первого наблюдения
df['days_since_first'] = (df['date'] - first_date).dt.days

# Удаляем ненужный столбец date
df = df.drop(columns=['date'])

## Второй признак

df['level_to_levels'] = (df['level'] / df['levels'])

In [ ]:
backup2 = pd.HDFStore('backup2.h5')
backup2['df'] = df
backup2.close()

In [ ]:
backup2 = pd.HDFStore('backup2.h5')
df = backup2['df']
backup2.close()

In [ ]:
from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# e_df = pd.DataFrame(scaler.fit_transform(df[["price", "level", "levels", "rooms", "area", "kitchen_area"]]))
# pd = pd.concat([boston, outliers], ignore_index = True)
# e_df.columns = scaler.get_feature_names_out()
# df = df.join(e_df)


# scaler = StandardScaler()
# e_df = pd.DataFrame(scaler.fit_transform(df[["price", "level", "levels", "rooms", "area", "kitchen_area"]]))
# print(e_df[:3])
# print(df[:3])
# e_df.columns = scaler.get_feature_names_out(df[["price", "level", "levels", "rooms", "area", "kitchen_area"]])
# df = df.join(e_df)


scaled_features = df.copy()

# Don't include the Name column in the transformation:

col_names = ["price", "level", "levels", "rooms", "area", "kitchen_area"]
features = scaled_features[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)

# Now, don't create a new dataframe but assign the result to those two columns:

scaled_features[col_names] = features
df = scaled_features.copy()
# print(scaled_features)



# Инициализация OneHotEncoder
# encoder = OneHotEncoder(sparse_output=False)

# # Применение OneHotEncoder к колонке building_type
# encoded_df = pd.DataFrame(encoder.fit_transform(df[['building_type']]))

# # Создание датафрейма с кодированными значениями
# encoded_df.columns = encoder.get_feature_names_out()
# #encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(['building_type']))

# # Объединение с оригинальным датафреймом
# df = df.join(encoded_df)
# df.drop('building_type', axis = 1, inplace = True)


In [ ]:
backup3 = pd.HDFStore('backup3.h5')
backup3['df'] = df
backup3.close()

In [ ]:
backup3 = pd.HDFStore('backup3.h5')
df = backup3['df']
backup3.close()

**2 Балла**. Реализуйте класс KNNRegressor, который должен делать регрессию методом k ближайших соседей.

In [ ]:
from sklearn.metrics import pairwise_distances

class KNNRegressor:
    def __init__(self, n_neighbors=5, metric='euclidean'):
        self.n_neighbors = n_neighbors
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        distances = pairwise_distances(X, self.X_train, metric=self.metric)
        neighbors_indices = np.argsort(distances, axis=1)[:, :self.n_neighbors]
        predictions = np.mean(self.y_train[neighbors_indices], axis=1)
        return predictions

**3 Балла**. Реализуйте класс LinearRegression, поддерживающий обучение градиентными спусками SGD, Momentum, AdaGrad. Используйте градиент для оптимизации функции потерь MSE.

In [ ]:
class LinearRegression:
    def __init__(self, learning_rate=0.01, optimization='SGD', epsilon=1e-8, decay_rate=0.9, max_iter=1000):
        self.learning_rate = learning_rate
        self.optimization = optimization
        self.epsilon = epsilon
        self.decay_rate = decay_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = None
        self.velocity = None
        self.gradient_squared = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        self.velocity = np.zeros(n_features)
        self.gradient_squared = np.zeros(n_features)

        for i in range(self.max_iter):
            # Прямой проход для предсказания
            y_pred = np.dot(X, self.weights) + self.bias

            # Вычисление ошибки
            error = y_pred - y
            mse_loss = np.mean(error ** 2)

            # Вычисление градиентов
            dw = (1 / n_samples) * np.dot(X.T, error)
            db = (1 / n_samples) * np.sum(error)

            if self.optimization == 'SGD':
                # Обновление весов для SGD
                self.weights = self.weights - (self.learning_rate * dw)
                self.bias = self.bias - (self.learning_rate * db)

            elif self.optimization == 'Momentum':
                # Обновление весов с использованием метода Momentum
                self.velocity = self.decay_rate * self.velocity + self.learning_rate * dw
                self.weights -= self.velocity
                self.bias -= self.learning_rate * db

            elif self.optimization == 'AdaGrad':
                # Обновление весов с использованием метода AdaGrad
                self.gradient_squared += dw ** 2
                adjusted_learning_rate = self.learning_rate / (np.sqrt(self.gradient_squared) + self.epsilon)
                self.weights -= adjusted_learning_rate * dw
                self.bias -= adjusted_learning_rate * db

            # Вывод потерь в процессе обучения (по желанию)
            if i % 100 == 0:
                print(f"Iteration {i}, MSE Loss: {mse_loss}")

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

# Часть 2. Эксперименты с моделями машинного обучения.

**3 Балла**. Проведите эксперименты с написанными Вами методами машинного обучения. Выделите обучающую и тестовую выборки в отношении 0,8 и 0,2 соответственно. Измерьте ошибку MSE, MAE, RMSE. Заиспользуйте методы KNNRegressor и LinearRegression из библиотеки sklearn, сравните качество Ваших решений и библиотечных.

In [ ]:
print(df[:5])

      price     level    levels     rooms      area  kitchen_area  is_Moscow  \
0 -0.394322  1.703464  2.706309 -0.559432 -0.944008      0.151638      False   
1 -0.547332 -0.229476 -0.864254 -0.559432 -0.835292      0.307421      False   
2  0.866173 -0.422770  0.234381  1.077090  1.258506      0.463204      False   
3 -0.295193 -0.616064 -0.864254  1.077090  1.137710      0.385313      False   
4 -0.386880 -0.809358 -1.138913 -0.559432 -0.956088      0.385313      False   

   is_Saint_Peterburg  building_type_0  building_type_1  building_type_2  \
0               False              1.0              0.0              0.0   
1               False              1.0              0.0              0.0   
2               False              0.0              0.0              0.0   
3               False              1.0              0.0              0.0   
4               False              0.0              0.0              0.0   

   building_type_3  building_type_4  building_type_5  building

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor as SklearnKNN
from sklearn.linear_model import LinearRegression as SklearnLinearRegression

# Разделим данные на признаки и целевую переменную
X = df.drop(columns='price').values  # Признаки
y = df['price'].values  # Целевая переменная

# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Обучение и оценка KNNRegressor
knn_regressor = KNNRegressor(n_neighbors=5)
knn_regressor.fit(X_train, y_train)
knn_predictions = knn_regressor.predict(X_test)

# Обучение и оценка LinearRegression
linear_regressor = LinearRegression()
linear_regressor.fit(X_train, y_train)
linear_predictions = linear_regressor.predict(X_test)

# Обучение и оценка sklearn KNN
sklearn_knn = SklearnKNN(n_neighbors=5)
sklearn_knn.fit(X_train, y_train)
sklearn_knn_predictions = sklearn_knn.predict(X_test)

# Обучение и оценка sklearn Linear Regression
sklearn_linear = SklearnLinearRegression()
sklearn_linear.fit(X_train, y_train)
sklearn_linear_predictions = sklearn_linear.predict(X_test)

# Вычисление ошибок
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mse, mae, rmse

# Оценка моделей
metrics = {
    "My KNN": evaluate_model(y_test, knn_predictions),
    "My Linear Regression": evaluate_model(y_test, linear_predictions),
    "Sklearn KNN": evaluate_model(y_test, sklearn_knn_predictions),
    "Sklearn Linear Regression": evaluate_model(y_test, sklearn_linear_predictions),
}

# Вывод результатов
for model, (mse, mae, rmse) in metrics.items():
    print(f"{model} - MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}")

Iteration 0, MSE Loss: 1.1078937862571514
Iteration 100, MSE Loss: 0.5968129599676274
Iteration 200, MSE Loss: 0.5483521273876467
Iteration 300, MSE Loss: 0.5263811911734636
Iteration 400, MSE Loss: 0.51249376352852
Iteration 500, MSE Loss: 0.5028277736633376
Iteration 600, MSE Loss: 0.4956049180311712
Iteration 700, MSE Loss: 0.48990936766196186
Iteration 800, MSE Loss: 0.4852440926922215
Iteration 900, MSE Loss: 0.4813232877063165
My KNN - MSE: 0.4190, MAE: 0.3143, RMSE: 0.6473
My Linear Regression - MSE: 0.3683, MAE: 0.3910, RMSE: 0.6069
Sklearn KNN - MSE: 0.4190, MAE: 0.3143, RMSE: 0.6473
Sklearn Linear Regression - MSE: 0.3048, MAE: 0.3789, RMSE: 0.5521
